# Test LLM use with Gigachat

In [1]:
CREDENTIALS = 'Yjg4MTQzMmUtNDAwMS00NDk0LThjOGUtNmU5ZWQ2YzQ4NDQ2OmQ4MWMxZGZiLTFmNGYtNDk5NS05OGQzLTBiMzYyYWJmNjk3OA=='
TESTPDF = "../data/papers/2014/2014_21.pdf"

In [2]:
from langchain.chat_models.gigachat import GigaChat
from langchain.chains.question_answering import load_qa_chain
from PyPDF2 import PdfReader

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter, CharacterTextSplitter
)

## 1. Test that Gigachat works

In [3]:
# Авторизация в сервисе GigaChat
giga = GigaChat(credentials=CREDENTIALS, 
                verify_ssl_certs=False,
                scope='GIGACHAT_API_CORP',
                model="GigaChat-Pro",
                )

## Recreate PDF retrieval with prompt and some test pdf

In [4]:
def extract_raw_text_from_pdf(path) -> str:
    reader = PdfReader(stream=path)

    raw_text = ''
    for _, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text

    return raw_text

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    # separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

raw_text = extract_raw_text_from_pdf(path=TESTPDF)
texts = text_splitter.split_text(text=raw_text)
embeddings = GigaChatEmbeddings(credentials=CREDENTIALS, verify_ssl_certs=False, scope='GIGACHAT_API_CORP')
index = FAISS.from_texts(texts=texts, embedding=embeddings)
chain = load_qa_chain(llm=giga, chain_type="stuff")

In [6]:
chain

StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Используй следующие части контекста, чтобы ответить на вопрос пользователя. \nЕсли ты не знаешь ответа, просто скажи, что не знаешь, не пытайся придумать ответ.\n----------------\n{context}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=GigaChat(credentials='Yjg4MTQzMmUtNDAwMS00NDk0LThjOGUtNmU5ZWQ2YzQ4NDQ2OmQ4MWMxZGZiLTFmNGYtNDk5NS05OGQzLTBiMzYyYWJmNjk3OA==', scope='GIGACHAT_API_CORP', model='GigaChat-Pro', verify_ssl_certs=False)), document_variable_name='context')

In [66]:
query = 'Please find HTM layers that were used in this paper. List them in the list. Example: [Spiro-OMeTAD, BPLA, TCP, ...]. If None found write [N/A]'
docs = index.similarity_search(query)
obj_list = chain.invoke({"input_documents": docs, "question": query})["output_text"]

In [67]:
query = "For each of these {Spiro HTM, Spiro-CB, Spiro-THF} report efficiency (PCE or optimized efficiency or η). Put that data in the markdown table with columns 'HTM' - 'PCE, %'"
docs = index.similarity_search(query)
table = chain.invoke({"input_documents": docs, "question": query})['output_text']

In [68]:
print(table)

| HTM       | PCE, %     |
|------------|------------|
| Spiro HTM  | Not reported    |
| Spiro-CB   | 15.27      |
| Spiro-THF  | 16.94      |


In [69]:
import mdpd

In [70]:
mdpd.from_md(table)

,HTM,"PCE, %"
0,Spiro HTM,Not reported
1,Spiro-CB,15.27
2,Spiro-THF,16.94
